# Inferential Statistics Ib - Frequentism

## Learning objectives

Welcome to the second Frequentist inference mini-project! Over the course of working on this mini-project and the previous frequentist mini-project, you'll learn the fundamental concepts associated with frequentist inference. The following list includes the topics you will become familiar with as you work through these two mini-projects:
* the _z_-statistic
* the _t_-statistic
* the difference and relationship between the two
* the Central Limit Theorem, its assumptions and consequences
* how to estimate the population mean and standard deviation from a sample
* the concept of a sampling distribution of a test statistic, particularly for the mean
* how to combine these concepts to calculate confidence intervals and p-values
* how those confidence intervals and p-values allow you to perform hypothesis (or A/B) tests

## Prerequisites

* what a random variable is
* what a probability density function (pdf) is
* what the cumulative density function is
* a high-level sense of what the Normal distribution

If these concepts are new to you, please take a few moments to Google these topics in order to get a sense of what they are and how you might use them.

These two notebooks were designed to bridge the gap between having a basic understanding of probability and random variables and being able to apply these concepts in Python. This second frequentist inference mini-project focuses on a real-world application of this type of inference to give you further practice using these concepts. 

In the previous notebook, we used only data from a known normal distribution. You'll now tackle real data, rather than simulated data, and answer some relevant real-world business problems using the data.

## Hospital medical charges

Imagine that a hospital has hired you as their data analyst. An administrator is working on the hospital's business operations plan and needs you to help them answer some business questions. This mini-project, as well as the bootstrap and Bayesian inference mini-projects also found in this unit are designed to illustrate how each of the inferential statistics methods have their uses for different use cases. In this assignment notebook, you're going to use frequentist statistical inference on a data sample to answer the questions:
* has the hospital's revenue stream fallen below a key threshold?
* are patients with insurance really charged different amounts than those without?
Answering that last question with a frequentist approach makes some assumptions, or requires some knowledge, about the two groups. In the next mini-project, you'll use bootstrapping to test that assumption. And in the final mini-project of the unit, you're going to create a model for simulating _individual_ charges (not a sampling distribution) that the hospital can use to model a range of scenarios.

We are going to use some data on medical charges obtained from [Kaggle](https://www.kaggle.com/easonlai/sample-insurance-claim-prediction-dataset). For the purposes of this exercise, assume the observations are the result of random sampling from our one hospital. Recall in the previous assignment, we introduced the Central Limit Theorem (CLT), and how it tells us that the distributions of sample statistics approach a normal distribution as $n$ increases. The amazing thing about this is that it applies to the sampling distributions of statistics that have been calculated from even highly non-normal distributions of data. Remember, also, that hypothesis testing is very much based on making inferences about such sample statistics. You're going to rely heavily on the CLT to apply frequentist (parametric) tests to answer the questions in this notebook.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t, ttest_ind_from_stats
from numpy.random import seed
sns.set()
sns.set_style({'axes.grid' : False})

medical = pd.read_csv('data/insurance2.csv')

In [2]:
medical.shape

(1338, 8)

In [3]:
medical.head()

,age,sex,bmi,children,smoker,region,charges,insuranceclaim
0,19,0,27.900,0,1,3,16884.92400,1
1,18,1,33.770,1,0,2,1725.55230,1
2,28,1,33.000,3,0,2,4449.46200,0
3,33,1,22.705,0,0,1,21984.47061,0
4,32,1,28.880,0,0,1,3866.85520,1


__Q:__ Plot the histogram of charges and calculate the mean and standard deviation. Comment on the appropriateness of these statistics for the data.

__A:__

In [6]:
_ = plt.hist( medical.charges, bins = 50 )
_ = plt.xlabel( "Amount Charged" )
_ = plt.ylabel("No: of people")

In [7]:
sample_mean = np.mean(medical.charges)
sample_std = np.std(medical.charges, ddof=1)
(sample_mean, sample_std)

# medical.charges.std()

(13270.422265141257, 12110.011236693994)

__Q:__ The administrator is concerned that the actual average charge has fallen below 12000, threatening the hospital's operational model. On the assumption that these data represent a random sample of charges, how would you justify that these data allow you to answer that question? And what would be the most appropriate frequentist test, of the ones discussed so far, to apply?

__A:__ The data is not normal and is rightly skewed. Still, we can use this data as the sample size which is greater than 1300 is quite large.

__Q:__ Given the nature of the administrator's concern, what is the appropriate confidence interval in this case? A one-sided or two-sided interval? Calculate the critical value and the relevant 95% confidence interval for the mean and comment on whether the administrator should be concerned?

__A:__ A one sided confidence interval is more suited here since the question is also one sided, that is, whether the mean is below 12000. The Null Hypotheis H0: is that we do not see a change in the sample mean (13270.42)

In [10]:
#Calculating critical value for mean being below 12000

p = 0.05
sample_size = medical.shape[0]
df = sample_size - 1

t_critical = t.ppf( p, df )

print(round(t_critical, 2))

-1.65


In [14]:
#Margin of error calculation

standard_error = sample_std / np.sqrt( sample_size )
margin = t_critical * standard_error

print( f"The margin of error is: {margin}"  )

lower_bound = sample_mean + margin

print( f"The lower bound of the confidence interval is: {lower_bound}" )

The margin of error is: -544.9350813250253
The lower bound of the confidence interval is: 12725.48718381623


As we can see that the lower bound is well above the 12000, the administor shouldn't be too concerned.

The administrator then wants to know whether people with insurance really are charged a different amount to those without.

__Q:__ State the null and alternative hypothesis here. Use the _t_-test for the difference between means where the pooled standard deviation of the two groups is given by
\begin{equation}
s_p = \sqrt{\frac{(n_0 - 1)s^2_0 + (n_1 - 1)s^2_1}{n_0 + n_1 - 2}}
\end{equation}

and the *t* test statistic is then given by

\begin{equation}
t = \frac{\bar{x}_0 - \bar{x}_1}{s_p \sqrt{1/n_0 + 1/n_1}}.
\end{equation}

What assumption about the variances of the two groups are we making here?

__A:__ The assumption we're making is that both the groups have same variance or in other words, are homogenous in nature.

The Null Hypothesis H0: That people with insurance are charged the same as people without insurance.

The Alternative Hypothesis H1: That people with insurance are charged differently from people without insurance.

__Q:__ Perform this hypothesis test both manually, using the above formulae, and then using the appropriate function from [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-tests) (hint, you're looking for a function to perform a _t_-test on two independent samples). For the manual approach, calculate the value of the test statistic and then its probability (the p-value). Verify you get the same results from both.

__A:__ 

### Manual Approach

In [30]:
x0 = medical[ medical.insuranceclaim == 0 ].charges.mean()
n0 = medical[ medical.insuranceclaim == 0 ].shape[0]
s0 = medical[ medical.insuranceclaim == 0 ].charges.std()
x1 = medical[ medical.insuranceclaim == 1 ].charges.mean()
n1 = medical[ medical.insuranceclaim == 1 ].shape[0]
s1 = medical[ medical.insuranceclaim == 1 ].charges.std()

sp = np.sqrt( ( (n0-1) * (s0**2) + (n1-1) * (s1**2) ) / ( n0 + n1 -2 ) )

In [33]:
t_critical = ( x0 - x1 ) / ( sp * np.sqrt( 1/n0 + 1/n1 ) )

In [34]:
p_value = t.cdf( t_critical, n0+n1-2 )

print( f"The critical t-statistic obtained manually is equal to: {t_critical}" )
print( f"The p_value that the null hypothesis is true is: {p_value}" )

The critical t-statistic obtained manually is equal to: -11.89329903087671
The p_value for the above Hypothesis Testing is equal to: 2.230615115810486e-31


### Using SciPy Module

In [36]:
t_critical_stats, p_value_stats = ttest_ind_from_stats( x0, s0, n0, x1, s1, n1 )

print( f"Using SciPy stats module, the t_critical value obtained is: {t_critical_stats} and the p_value is: {p_value_stats}" )

Using SciPy stats module, the t_critical value obtained is: -11.893299030876712 and the p_value is: 4.461230231620717e-31


Congratulations! Hopefully you got the exact same numerical results. This shows that you correctly calculated the numbers by hand. Secondly, you used the correct function and saw that it's much easier to use. All you need to do pass your data to it.

__Q:__ In the above calculations, we assumed the sample variances were equal. We may well suspect they are not (we'll explore this in another assignment). The calculation becomes a little more complicated to do by hand in this case, but we now know of a helpful function. Check the documentation for the function to tell it not to assume equal variances and perform the test again.

__A:__ If the two groups have unequal variances, that is, they're heteroskedastic, then we perform Welch Test.

In [37]:
t_critical_welch, p_value_welch = ttest_ind_from_stats( x0, s0, n0, x1, s1, n1, equal_var=False )

In [38]:
print( f"Using Welch Test, the t_critical value obtained is: {t_critical_welch} and the p_value is: {p_value_welch}" )

Using Welch Test, the t_critical value obtained is: -13.298031957975647 and the p_value is: 1.1105103216309438e-37


__Q:__ Conceptual question: look through the documentation for statistical test functions in scipy.stats. You'll see the above _t_-test for a sample, but can you see an equivalent one for performing a *z*-test from a sample? Comment on your answer.

__A:__ I was not able to find a function to perform z-test on a sample in scipy.stats

## Learning outcomes

Having completed this project notebook, you now have good hands-on experience:
* using the central limit theorem to help you apply frequentist techniques to answer questions that pertain to very non-normally distributed data from the real world
* performing inference using such data to answer business questions
* forming a hypothesis and framing the null and alternative hypotheses
* testing this using a _t_-test